In [1]:
#include <vector>
#include <iostream>
#include <string>
#include <algorithm>
#include <memory>

In [2]:
template <typename T>
std::ostream & operator << (std::ostream & os, const std::vector<T> & vec)
{
    for(auto elem : vec)
    {
        os<<elem<< " ";
    }
    return os;
}

## 1 create a vector from an existing vector: call copy constructor

In [3]:
std::vector<std::string> a_vector = {"a","ccc","ddd","e"};

In [4]:
std::vector<std::string> another(a_vector);

In [5]:
std::cout << another << std::endl;

a ccc ddd e 


## 2 create a vector from last elements of an existing vector

In [6]:
std::int64_t n = 3;

In [7]:
std::cout << std::min(int(n),int(a_vector.size())); 

3

In [8]:
std::vector<std::string> last_n_vector(a_vector.end()-std::min(int(n),int(a_vector.size())), a_vector.end());

In [9]:
std::cout << last_n_vector << std::endl;

ccc ddd e 


## 3 concat two vector

In [10]:
std::vector<int> x = { 1, 2, 7};
std::vector<int> y = { 4, 5 };
 
// 1. Copy constructor + vector::insert
 
std::vector<int> v(x);
v.insert(v.end(), y.begin(), y.end());

In [11]:
std::cout << v << std::endl;

1 2 7 4 5 


# appendix 1: shared_ptr with vector

https://stackoverflow.com/questions/28733385/c11-vector-of-smart-pointer

In [12]:
class Base {
    public:
    virtual void f() { std::cout << "Base::f()" << std::endl;}
    //virtual ~Base() = default; // to silence -Wall warnings
}

In [13]:
class Derived: public Base {
    void f() override { std::cout << "Derived::f()" << std::endl;}
};

In [14]:
{
    std::vector<std::shared_ptr<Base>> vsp; // access Derived via shared_ptr to Base

    std::shared_ptr<Base> base = std::make_shared<Base>();
    std::shared_ptr<Derived> derived = std::make_shared<Derived>();

    vsp.push_back(base);
    vsp.push_back(derived);

    for(auto&& elem: vsp)
        elem->f(); // virtual dispatch
}

Base::f()
Derived::f()


## appendix 2: access std::unique_ptr element from vector

https://stackoverflow.com/questions/20773351/error-after-trying-to-get-unique-ptr-element-from-vector

https://stackoverflow.com/questions/47478481/access-unique-ptr-elements-in-container

### take ownership solution

unique_ptr cannot be copied or assigned. It needs to be move copied or move assigned. You can achieve this by use of std::move:

~~~
std::unique_ptr<A> a = std::move(vec_a.at(0));
~~~

The call to std::move essentially allows you to treat the RHS as an rvalue, which can then be used in unique_ptr'a move copy constructor.

Note that after this, you will have to be careful with the first element of vec_a.

### not take ownership solution

#### approch 1:  get the raw pointer from the unique_ptr
If you don't want to take ownership of the pointer away from the vector, simply access the managed pointer directly:

In modern C++ raw pointers denote non-owning relationships, so use xxx.get() to get the raw pointer from the unique_ptr

~~~
A* a = vec_a.at(0).get();
~~~
**You have to be careful not to use a after the object it points to has ceased to exist.**

#### approch 2: get a reference to the pointed-to object
If you are certain that all objects in the container are non-null, then obtaining a reference to the pointed-to object is a perfectly valid approach.
~~~
const A& pointed_to_object_reference = *vec_a.at(0);
~~~

#### approch 3: get a reference to the unique_ptr itself
Obtain a reference to the unique_ptr itself, which allows you to assign a value to it in the event you want to change the object it points to.
~~~
std::unique_ptr<A>& unique_ptr_reference = vec_a.at(0);
~~~


In [15]:
{
  std::vector<std::unique_ptr<Base>> vec_a;
  std::unique_ptr<Base> tmp_a = std::unique_ptr<Base>(new Base());

  vec_a.push_back(std::move(tmp_a));

  vec_a.push_back(std::unique_ptr<Base>(new Base()));
    
  // how to access the unique_ptr from a vector?
  std::unique_ptr<Base> a = std::move(vec_a.at(0));
  a->f();
  
  // check nullptr
  if (static_cast<bool>(vec_a.at(0))){
      std::cout << "not null" << std::endl;
  } else {
      std::cout << "is null" << std::endl;
  }
    
  // another approch to check    
  if (vec_a.at(0)){
      std::cout << "vec_a.at(0) is not a nullptr" << std::endl; 
  } else {
      std::cout << "vec_a.at(0) is a nullptr now" << std::endl;
  }
      
  Base* raw_ptr = vec_a.at(1).get();
  raw_ptr->f();
  
  if (static_cast<bool>(vec_a.at(1))){
      std::cout << "not null" << std::endl;
  } else {
      std::cout << "is null" << std::endl;
  }

  Base& pointed_to_object_reference = *vec_a.at(1);
  pointed_to_object_reference.f();
  
  std::unique_ptr<Base>& unique_ptr_reference = vec_a.at(1);
  unique_ptr_reference->f();
}

Base::f()
is null
vec_a.at(0) is a nullptr now
Base::f()
not null
Base::f()
Base::f()
